In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Carregar os dados
dados = pd.read_csv("Dados Históricos - Ibovespa (2).csv")

# Ajustar nomes das colunas (remover espaços e caracteres especiais)
dados.columns = [col.strip().replace('.', '').replace('%', '').replace(' ', '_') for col in dados.columns]

# Converter colunas numéricas (trocar vírgula por ponto e transformar em float)
for col in ['Último', 'Abertura', 'Máxima', 'Mínima']:
    dados[col] = dados[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# Ordenar por data crescente
dados['Data'] = pd.to_datetime(dados['Data'], dayfirst=True)
dados = dados.sort_values('Data').reset_index(drop=True)

# Criar coluna de variação do índice
dados['Variação'] = dados['Último'].diff().fillna(0)

# Novas features
dados['Var_pct'] = dados['Último'].pct_change().fillna(0)
dados['MM3'] = dados['Último'].rolling(window=3).mean().fillna(method='bfill')
dados['Range'] = dados['Máxima'] - dados['Mínima']

# Definir as categorias de variação
def classifica_variacao(x):
    if x > 0:
        return 'Subiu'
    elif x < 0:
        return 'Caiu'
    else:
        return 'Manteve'

dados['Alvo'] = dados['Variação'].apply(classifica_variacao)

# Defasar as features em 1 dia
features = ['Último', 'Abertura', 'Máxima', 'Mínima', 'Var_pct', 'MM3', 'Range']
for f in features:
    dados[f + '_lag1'] = dados[f].shift(1)

# Remover linhas iniciais com NaN
dados = dados.iloc[1:].reset_index(drop=True)

# Remover colunas não usadas
dados = dados.drop(columns=['Vol', 'Var', 'Data', 'Variação'] + features)

# Separar features e target
x = dados.drop(columns=['Alvo'])
y = dados['Alvo']

# Separar em treino e teste (sem embaralhar para respeitar ordem temporal)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7, shuffle=False)

# Treinar o classificador RandomForest
rf = RandomForestClassifier(random_state=7, class_weight='balanced')
rf.fit(x_train, y_train)

# Avaliação do modelo
y_predito = rf.predict(x_test)
print("Acurácia:", accuracy_score(y_test, y_predito))
print("\nClassification Report:\n", classification_report(y_test, y_predito))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_predito))

# Previsão dos próximos dias
dias_a_prever = 10  # Altere aqui para quantos dias quiser prever

ultimos_dados = dados.copy()
previsoes = []
simulados = []

for i in range(dias_a_prever):
    ultima = ultimos_dados.iloc[[-1]].copy()
    variacao = random.uniform(-0.04, 0.04)
    proximo_ultimo = ultima['Último_lag1'].values[0] * (1 + variacao)
    abertura = proximo_ultimo * (1 + random.uniform(-0.015, 0.015))
    maxima = proximo_ultimo * (1 + abs(random.uniform(0, 0.03)))
    minima = proximo_ultimo * (1 - abs(random.uniform(0, 0.03)))
    var_pct = (proximo_ultimo - ultima['Último_lag1'].values[0]) / ultima['Último_lag1'].values[0]
    mm3 = np.mean([proximo_ultimo, ultima['Último_lag1'].values[0], ultima['Último_lag1'].values[0]])
    range_ = maxima - minima

    nova_linha = pd.DataFrame({
        'Último_lag1': [proximo_ultimo],
        'Abertura_lag1': [abertura],
        'Máxima_lag1': [maxima],
        'Mínima_lag1': [minima],
        'Var_pct_lag1': [var_pct],
        'MM3_lag1': [mm3],
        'Range_lag1': [range_]
    })

    pred = rf.predict(nova_linha)[0]
    previsoes.append(pred)
    simulados.append([proximo_ultimo, abertura, maxima, minima])

    nova_linha_full = ultima.copy()
    for col, val in zip(['Último_lag1', 'Abertura_lag1', 'Máxima_lag1', 'Mínima_lag1', 'Var_pct_lag1', 'MM3_lag1', 'Range_lag1'],
                        [proximo_ultimo, abertura, maxima, minima, var_pct, mm3, range_]):
        nova_linha_full[col] = val
    ultimos_dados = pd.concat([ultimos_dados, nova_linha_full], ignore_index=True)

print(f"\nPrevisão para os próximos {dias_a_prever} dias:")
for i, (p, s) in enumerate(zip(previsoes, simulados), 1):
    print(f"Dia {i}: {p} | Último: {s[0]:.2f} | Abertura: {s[1]:.2f} | Máxima: {s[2]:.2f} | Mínima: {s[3]:.2f}")

Acurácia: 0.63

Classification Report:
               precision    recall  f1-score   support

        Caiu       0.57      0.76      0.65        46
       Subiu       0.72      0.52      0.60        54

    accuracy                           0.63       100
   macro avg       0.65      0.64      0.63       100
weighted avg       0.65      0.63      0.63       100

Matriz de Confusão:
 [[35 11]
 [26 28]]

Previsão para os próximos 10 dias:
Dia 1: Caiu | Último: 139931.06 | Abertura: 139669.52 | Máxima: 142771.73 | Mínima: 139153.77
Dia 2: Caiu | Último: 139273.66 | Abertura: 137323.02 | Máxima: 143417.63 | Mínima: 138365.98
Dia 3: Subiu | Último: 135457.81 | Abertura: 134773.82 | Máxima: 137016.38 | Mínima: 134451.66
Dia 4: Caiu | Último: 131506.88 | Abertura: 131205.27 | Máxima: 134147.51 | Mínima: 127635.95
Dia 5: Caiu | Último: 126993.70 | Abertura: 128024.84 | Máxima: 129189.74 | Mínima: 126378.52
Dia 6: Caiu | Último: 128837.14 | Abertura: 128216.12 | Máxima: 129630.36 | Mínima: 12